In [0]:
import pickle as pk
# from google.colab import drive
import os

import tensorflow as tf
import numpy as np
from time import time

# drive.mount('/content/gdrive', force_remount=True)


In [2]:
!git clone https://github.com/RafaelDrumond/PeekDB.git

Cloning into 'PeekDB'...
remote: Enumerating objects: 86, done.
remote: Total 86 (delta 0), reused 0 (delta 0), pack-reused 86
Unpacking objects: 100% (86/86), done.


In [3]:
!mkdir -p val
!cp ./PeekDB/Motion/Actor2* ./val/
!ls ./val/

Actor2Crouching.txt  Actor2Running.txt	Actor2Walking.txt
Actor2Idle.txt	     Actor2Swing.txt


In [4]:
!mkdir -p tr
!cp ./PeekDB/Motion/Actor* ./tr/
!rm ./tr/Actor2*
!rm ./tr/Actor0*
!ls ./tr/

Actor10Crouching.txt  Actor4Crouching.txt  Actor7Crouching.txt
Actor10Idle.txt       Actor4Idle.txt	   Actor7Idle.txt
Actor10Running.txt    Actor4Running.txt    Actor7Running.txt
Actor10Swing.txt      Actor4Swing.txt	   Actor7Swing.txt
Actor10Walking.txt    Actor4Walking.txt    Actor7Walking.txt
Actor1Crouching.txt   Actor5Crouching.txt  Actor8Crouching.txt
Actor1Idle.txt	      Actor5Idle.txt	   Actor8Idle.txt
Actor1Running.txt     Actor5Running.txt    Actor8Running.txt
Actor1Swing.txt       Actor5Swing.txt	   Actor8Swing.txt
Actor1Walking.txt     Actor5Walking.txt    Actor8Walking.txt
Actor3Crouching.txt   Actor6Crouching.txt  Actor9Crouching.txt
Actor3Idle.txt	      Actor6Idle.txt	   Actor9Idle.txt
Actor3Running.txt     Actor6Running.txt    Actor9Running.txt
Actor3Swing.txt       Actor6Swing.txt	   Actor9Swing.txt
Actor3Walking.txt     Actor6Walking.txt    Actor9Walking.txt


In [0]:

def motionLoad(classes, directory, window = 60):

  if directory.endswith("/"):
    dire = directory
  else:
    dire = directory+"/"
  file_list = os.listdir(dire)
#   print(file_list)
  file_list = [i for i in file_list if any(s in i for s in classes)]
#   print(file_list)
  instances = []
  labels = []
  #READ FILES
  for f in file_list:
    label = [0.,0.,0.]
    index = 0
    #CHECK CLASS
    for c, cla in enumerate(classes):
      if cla in f:
        label[c] = 1.0
    #OPEN FILE
    with open(directory+f) as doc:
      lines = doc.readlines()
    end = 0
    #READ AND APPEND
    while index < len(lines):
      count = 0
      subin = []
      if index+window < len(lines):
        while count < window:
          count +=1
          out = lines[index].split("|m ")[1].split(" |class")[0].split(" ") # if you make changes look here
          out = [float(i) for i in out]
          subin.append(out)
          index +=1 # pay attention here!
        instances.append(np.array(subin))
        labels.append   (np.array(label))
      else:
        index = len(lines)
  return instances, labels

In [0]:
classes = ["Crouching", "Idle", "Running", "Siwing", "Walking"]
actions = ["Crouching", "Running", "Swing"]
window  = 40
n_actions = len(actions)

xv, yv = motionLoad(actions, "./val/", window)
xt, yt = motionLoad(["Crouching", "Running", "Swing"], "./tr/", window)

In [7]:
n_tr = len(xt)
n_ts = len(xv)
print(n_tr, n_ts)

4816 532


In [8]:
x = tf.placeholder(tf.float32, [None, window, 20], name="Inputs")
y = tf.placeholder(tf.float32, [None, n_actions], name="Labels")

model = tf.keras.layers.LSTM(200, name="LSTM1")(x)
model = tf.layers.Dropout(.5, name="dropout1")(model)
model = tf.layers.Dense(27, name="dense1")(model)
model = tf.layers.Dense(n_actions, name="dense2")(model)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=model), name="loss")

# optimizer
optimizer = tf.train.AdamOptimizer().minimize(loss)

# Accuracy
prediction = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name="accuracy")


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
epochs = 20
batch_size = 50
batches = n_tr // batch_size
print("Begin Session")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    counter = 0
    tic = time()
    for epoch in range(epochs):
        print("Start Epoch {:>2}".format(epoch+1))
        t = time()
        for i in range(batches):
            tloss, tacc, _ = sess.run([loss, accuracy, optimizer],
                                         feed_dict={
                                             x: xt[i*batch_size:(i+1)*batch_size], 
                                             y: yt[i*batch_size:(i+1)*batch_size]})
        print("Training   Loss: {:>8.4f}, Training   Accuracy: {:>8.6f}".format(tloss, tacc))
        print("Epoch time: ", time() - t)
    vloss, vacc = sess.run([loss, accuracy], feed_dict={x: xv, y: yv})
    print("Validation Loss: {:>8.4f}, Validation Accuracy: {:>8.6f}".format(vloss, vacc))
    print("Training time:", time() - tic)

Begin Session
Start Epoch  1
Training   Loss:   0.3956, Training   Accuracy: 0.920000
Epoch time:  24.22856092453003
Start Epoch  2
Training   Loss:   0.3146, Training   Accuracy: 0.860000
Epoch time:  23.47832441329956
Start Epoch  3
Training   Loss:   0.1872, Training   Accuracy: 0.900000
Epoch time:  23.26110029220581
Start Epoch  4
Training   Loss:   0.0645, Training   Accuracy: 1.000000
Epoch time:  23.149770975112915
Start Epoch  5
Training   Loss:   0.0391, Training   Accuracy: 1.000000
Epoch time:  23.177053213119507
Start Epoch  6
Training   Loss:   0.0883, Training   Accuracy: 0.940000
Epoch time:  23.173630952835083
Start Epoch  7
Training   Loss:   0.0388, Training   Accuracy: 1.000000
Epoch time:  23.092628240585327
Start Epoch  8
Training   Loss:   0.0197, Training   Accuracy: 1.000000
Epoch time:  23.734130859375
Start Epoch  9
Training   Loss:   0.0131, Training   Accuracy: 1.000000
Epoch time:  23.014073610305786
Start Epoch 10
Training   Loss:   0.0275, Training   Acc

KeyboardInterrupt: ignored

In [11]:
x = tf.placeholder(tf.float32, [None, window, 20], name="Inputs")
y = tf.placeholder(tf.float32, [None, n_actions], name="Labels")

model = tf.layers.Conv1D(filters=128, kernel_size=8, name="Conv1")(x)
model = tf.layers.BatchNormalization()(model)
model = tf.nn.relu(model)

model = tf.layers.Conv1D(filters=256, kernel_size=5, name="Conv2")(x)
model = tf.layers.BatchNormalization()(model)
model = tf.nn.relu(model)

model = tf.layers.Conv1D(filters=128, kernel_size=3, name="Conv3")(x)
model = tf.layers.BatchNormalization()(model)
model = tf.nn.relu(model)

model = tf.keras.layers.GlobalAveragePooling1D()(model)
model = tf.layers.Dense(n_actions, name="dense2")(model)


# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=model), name="loss")

# optimizer
optimizer = tf.train.AdamOptimizer().minimize(loss)

# Accuracy
prediction = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32), name="accuracy")

epochs = 20
batch_size = 20
batches = n_tr // batch_size
print("Begin Session")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    counter = 0
    tic = time()
    for epoch in range(epochs):
        print("Start Epoch {:>2}".format(epoch+1))
        t = time()
        for i in range(batches):
            tloss, tacc, _ = sess.run([loss, accuracy, optimizer],
                                         feed_dict={
                                             x: xt[i*batch_size:(i+1)*batch_size], 
                                             y: yt[i*batch_size:(i+1)*batch_size]})
        print("Training   Loss: {:>8.4f}, Training   Accuracy: {:>8.6f}".format(tloss, tacc))
        print("Epoch time: ", time() - t)
    vloss, vacc = sess.run([loss, accuracy], feed_dict={x: xv, y: yv})
    print("Validation Loss: {:>8.4f}, Validation Accuracy: {:>8.6f}".format(vloss, vacc))
    print("Training time:", time() - tic)


Instructions for updating:
Use tf.cast instead.
Begin Session
Start Epoch  1
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  1.195894479751587
Start Epoch  2
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  0.9991838932037354
Start Epoch  3
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  0.9755303859710693
Start Epoch  4
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  0.9592993259429932
Start Epoch  5
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  0.9776597023010254
Start Epoch  6
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  0.975836992263794
Start Epoch  7
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  0.9915745258331299
Start Epoch  8
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  0.9747292995452881
Start Epoch  9
Training   Loss:   0.0000, Training   Accuracy: 1.000000
Epoch time:  0.9810945987701416
Star